In [1]:
import pickle
from pathlib import Path

import pandas as pd
import torch
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    Trainer,
    TrainingArguments,
)

/home/kjqw/.conda/envs/transformers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# パスの管理
data_path = Path("data/data_long_texts.tsv")
satisfaction_model_path = Path("data/satisfaction_model")
label_model_path = Path("data/label_model")

# モデルの選択
MODEL_NAME = "cl-tohoku/bert-base-japanese-v3"
# MODEL_NAME = "google-bert/bert-base-multilingual-cased"

In [3]:
# データの読み込み
data = pd.read_csv(data_path, sep="\t")

# 満足度のエンコード（0: 不満, 1: 満足）
data["満足度"] = data["満足度"].map({"不満": 0, "満足": 1})

# データを訓練データとテストデータに分割
train_data, test_data = train_test_split(data, test_size=0.2)

# デバイスの設定
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
# モデルとトークナイザーのロード
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# 満足度分類モデルのロード
satisfaction_model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)
satisfaction_model.to(device)  # モデルをデバイスに移動

# ラベルのエンコード
label_encoder = LabelEncoder()
data["ラベル"] = label_encoder.fit_transform(data["ラベル"])
num_labels = len(label_encoder.classes_)

# ラベル分類モデルのロード
label_model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=num_labels
)
label_model.to(device)  # モデルをデバイスに移動

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
# データセットクラスの作成
class CustomDataset(Dataset):
    """
    データセットクラス

    Parameters
    ----------
    encodings : dict
        エンコーディングされたデータ
    labels : list
        ラベルのリスト

    Returns
    -------
    item : dict
        エンコーディングされたデータとラベルのペア
    """

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# データの前処理
def preprocess_data(data: pd.DataFrame, label_col: str) -> tuple:
    """
    データの前処理

    Parameters
    ----------
    data : pd.DataFrame
        データ
    label_col : str
        ラベルのカラム名

    Returns
    -------
    encodings : dict
        エンコーディングされたデータ
    labels : list
        ラベルのリスト
    """

    encodings = tokenizer(
        data["文章"].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt",
    )
    labels = data[label_col].tolist()
    return encodings, labels

In [6]:
# 満足度分類用データセットの作成
satisfaction_encodings, satisfaction_labels = preprocess_data(data, "満足度")
satisfaction_dataset = CustomDataset(satisfaction_encodings, satisfaction_labels)

# ラベル分類用データセットの作成
label_encodings, label_labels = preprocess_data(data, "ラベル")
label_dataset = CustomDataset(label_encodings, label_labels)

In [7]:
# 評価指標の計算関数
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [8]:
# トレーニング引数の設定
train_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
)

In [9]:
# 満足度分類モデルのトレーナーの作成
satisfaction_trainer = Trainer(
    model=satisfaction_model,
    args=train_args,
    train_dataset=satisfaction_dataset,
    eval_dataset=satisfaction_dataset,
    compute_metrics=compute_metrics,
)

In [10]:
# 満足度分類モデルのトレーニング
satisfaction_trainer.train()

/tmp/ipykernel_35931/1952686614.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=276, training_loss=0.48487439362899115, metrics={'train_runtime': 27.1161, 'train_samples_per_second': 80.874, 'train_steps_per_second': 10.178, 'total_flos': 144250636101120.0, 'train_loss': 0.48487439362899115, 'epoch': 3.0})

In [11]:
# モデルの保存
satisfaction_trainer.save_model(satisfaction_model_path)

In [12]:
# ラベル分類モデルのトレーナーの作成
label_trainer = Trainer(
    model=label_model,
    args=train_args,
    train_dataset=label_dataset,
    eval_dataset=label_dataset,
    compute_metrics=compute_metrics,
)

In [13]:
# ラベル分類モデルのトレーニング
label_trainer.train()

/tmp/ipykernel_35931/1952686614.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=276, training_loss=1.9978533758633379, metrics={'train_runtime': 27.0918, 'train_samples_per_second': 80.947, 'train_steps_per_second': 10.188, 'total_flos': 144268768456704.0, 'train_loss': 1.9978533758633379, 'epoch': 3.0})

In [14]:
# モデルの保存
label_trainer.save_model(label_model_path)

In [15]:
# テストデータの準備
test_texts = test_data["文章"].tolist()
test_satisfactions = ["満足" if i == 1 else "不満" for i in test_data["満足度"]]
test_labels = test_data["ラベル"].tolist()

# テストデータのトークナイズ
test_encodings = tokenizer(
    test_texts, truncation=True, padding=True, return_tensors="pt"
).to(device)

# モデルを評価モードに設定
satisfaction_model.eval()
label_model.eval()

# 予測の取得
with torch.no_grad():
    satisfaction_outputs = satisfaction_model(**test_encodings)
    satisfaction_preds = torch.argmax(satisfaction_outputs.logits, dim=1).cpu().numpy()

    label_outputs = label_model(**test_encodings)
    label_preds = torch.argmax(label_outputs.logits, dim=1).cpu().numpy()

# 数値から文字列に変換
label_result = label_encoder.inverse_transform(label_preds).tolist()
satisfaction_result = ["満足" if pred == 1 else "不満" for pred in satisfaction_preds]

In [16]:
df_compare_test_data = pd.DataFrame(
    {
        "満足度": test_satisfactions,
        "満足度予測": satisfaction_result,
        "満足度一致": [i == j for i, j in zip(test_satisfactions, satisfaction_result)],
        "ラベル": test_labels,
        "ラベル予測": label_result,
        "ラベル一致": [i == j for i, j in zip(test_labels, label_result)],
        "文章": test_texts,
    }
)

In [17]:
df_compare_test_data

,満足度,満足度予測,満足度一致,ラベル,ラベル予測,ラベル一致,文章
0,満足,満足,True,買物・飲食,買物・飲食,True,ほとんどの飲食チェーン店がある、ノクティ内ならノクティﾎﾟｲﾝﾄも付いてお得
1,満足,満足,True,買物・飲食,買物・飲食,True,自宅から５分以内にスーパーうあ個人商店が立ち並んでいる。
2,満足,満足,True,移動・交通,移動・交通,True,駅やバス停に近く、タクシー等もつかまえやすい
3,満足,満足,True,自然災害,自然災害,True,山崩れ・土砂崩れ、水害等の 災害を心配なくてよい。
4,不満,不満,True,デジタル生活,地域行政,False,市が率先して、デジタル生活を推し進める施策をしてほしい
...,...,...,...,...,...,...,...
142,不満,不満,True,住宅環境,住宅環境,True,一戸建てを購入したいので、良い住宅環境を望みます
143,不満,不満,True,移動・交通,移動・交通,True,とにかく道路の舗装や道路標示の質が悪すぎる。
144,不満,不満,True,医療・福祉,医療・福祉,True,病院、特に発熱外来など東京などに比べすぐに対応できるところが少ない。コロナ禍の時に感じました。
145,満足,満足,True,住宅環境,移動・交通,False,第二東名などの交通整備が進み、車での移動が楽。近隣市町村でのショッピングにも事欠かない。


In [18]:
with open("data/df_compare_test_data.pickle", "wb") as f:
    pickle.dump(df_compare_test_data, f)